In [8]:
import pandas as pd
import json
from matplotlib import pyplot as plt
from PIL import Image
import os.path

def to_bbox(bbox):
    return [bbox[0], bbox[1], bbox[2]+bbox[0], bbox[3]+bbox[1]]

def load_json(path):
    with open(path, "r") as file:
        json_object = json.load(file)
    return json_object
def save_json(path, obj):
    with open(path, "w") as file:
        json.dump(obj, file)
path = '/ekaterina/work/data/plain_zebra/annotations/preprocessed/plain_zebra_singletons_encounters_flipped.json'
data = load_json(path)
orig_path = '/ekaterina/work/data/plain_zebra/annotations/preprocessed/plain_zebra.json'
orig_data = load_json(orig_path)

def find_annotation_ind(annots, image_uuid, index="image_uuid"):
    for i,annot in enumerate(annots):
        if annot[index] == image_uuid:
            return i

img_path = '/ekaterina/work/data/plain_zebra/images/train2020'

for image in data["images"]:
    # if "xflipped" not in image["file_name"]:
    #     continue
    
    annot_i = find_annotation_ind(data["annotations"], image["uuid"])
    orig_annot_i = find_annotation_ind(orig_data["annotations"], image["uuid"])
    orig_individ_i = find_annotation_ind(orig_data["individuals"], orig_data["annotations"][orig_annot_i]["individual_uuid"], index="uuid")

    orig_img_i = find_annotation_ind(orig_data["images"], orig_data["annotations"][orig_annot_i]["image_uuid"], index="uuid")

    orig_viewpoint = orig_data["annotations"][orig_annot_i]["viewpoint"].replace("front", "").replace("back", "")
    orig_name = orig_data["individuals"][orig_individ_i]["name"]
    orig_bbox = orig_data["annotations"][orig_annot_i]["bbox"]

    orig_name_viewpoint = orig_name + "_" + orig_viewpoint
    
    # image["file_name"] = image["file_name"].replace("__xflipped", "")
    image["file_name"] = orig_data["images"][orig_img_i]['file_name']
    data["annotations"][annot_i]["bbox"] = orig_bbox
    data["annotations"][annot_i]["name"] = orig_name
    data["annotations"][annot_i]["viewpoint"] = orig_viewpoint
    data["annotations"][annot_i]["name_viewpoint"] = orig_name_viewpoint

    img_name = orig_data["images"][orig_img_i]['file_name']#image["file_name"]

    # print(data["annotations"][annot_i]["name_viewpoint"])
    # plt.figure(figsize=(20, 15))
    # plt.subplot(121)
    # plt.imshow(Image.open(os.path.join(img_path, img_name)))
    # bbox = to_bbox(data["annotations"][annot_i]["bbox"])
    # plt.plot([bbox[i] for i in [0, 2, 2, 0, 0]], [bbox[i] for i in [1, 1, 3, 3, 1]])
    # plt.subplot(122)
    # plt.imshow(Image.open(os.path.join(img_path, img_name)).crop(to_bbox(data["annotations"][annot_i]["bbox"])))
    # plt.show()

out_path = '/ekaterina/work/data/plain_zebra/annotations/preprocessed/plain_zebra_singletons_encounters_notflipped.json'
save_json(out_path, data)
